In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 0s (10.1 MB/s)
debconf: unable to initia

- Restart the runtime after executing the above cell and after restarting continue from the cell below

In [ ]:
# Importing all the required libraries

import spacy
import pytesseract as tess
import re
import cv2 as cv
spacy.__version__

'3.4.1'

In [ ]:
# uploading the files from local machine

from google.colab import files
uploaded = files.upload()

Saving bill.jpg to bill.jpg


In [ ]:
# importing image and extracting as a text

# accessing first value of 'uploaded' as it is the file name
key = list(uploaded.keys())[0]

img = cv.imread(key)  # create matrix of image
text = tess.image_to_string(img)  # extraction of data from image
text

'RECEIPT\n\n \n\n \n\nTerminal##t 01-02-2018 11:32AM\n1xT-Shirt $25.50\n1x Watches $299.00\n1x Pants $32.99\n1x Socks $6.50\nTOTAL AMOUNT $363.99\nCASH $400.00\nCHANGE $36.01\nBank Card FI IIE IE I91\nApprovaltt 192107\n\n \n\nsence THANK YOU! eens\n\nWIN\n\x0c'

In [ ]:
# Loading NLP model from spacy
nlp = spacy.load("en_core_web_sm")

doc = nlp(text)

In [ ]:
# extacting name, prices, cities, dates etc
names = []
prices = []
cities = []
dates = []

for ent in doc.ents:
    if ent.label_ == "PERSON":
        names.append(ent.text)

    elif ent.label_ == "MONEY":
        prices.append(ent.text) 

    elif ent.label_ == "GPE":
        cities.append(ent.text)

    elif ent.label_ == "DATE":
        dates.append(ent.text)

print(names,prices,cities,dates,sep="\n")

[]
['25.50', '299.00', '32.99', '6.50', '363.99', '400.00', '36.01']
[]
['01-02-2018', '192107']


In [ ]:
def extract_phone_numbers(string):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = r.findall(string)
    return [re.sub(r'\D', '', number) for number in phone_numbers]

def extract_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)

In [ ]:
# Extracting phone numbers and emails from text

phone = extract_phone_numbers(text)
emails = extract_email_addresses(text)

print(phone,emails,sep="\n")

[]
[]


In [ ]:
import json
data = {
    "names" : names,
    "prices": prices,
    "cities":cities,
    "dates" : dates,
    "phones" : phone,
    "emails" : emails
}

data_json = json.dumps(data)
data_json

'{"names": [], "prices": ["25.50", "299.00", "32.99", "6.50", "363.99", "400.00", "36.01"], "cities": [], "dates": ["01-02-2018", "192107"], "phones": [], "emails": []}'

In [ ]:
output = {}

for ent in doc.ents :
    if ent.label_ in output.keys():
        output[ent.label_].append(ent.text)
    else :
        output[ent.label_] = []
        output[ent.label_].append(ent.text)

output["phone"] = phone
output["emails"] = emails

output_2 = json.dumps(output)

print(output_2)

{"PRODUCT": ["RECEIPT"], "CARDINAL": ["Terminal##t", "11:32AM"], "DATE": ["01-02-2018", "192107"], "ORG": ["1xT-Shirt", "Bank Card FI IIE IE I91\nApprovaltt"], "MONEY": ["25.50", "299.00", "32.99", "6.50", "363.99", "400.00", "36.01"], "phone": [], "emails": []}


In [ ]:
with open("sample.json","w") as output_file :
  json.dump(output,output_file)

In [ ]:
data = json.load(open("sample.json"))

final_output_data = {
    "names" : data['MONEY'],
    "prices": data["MONEY"],
    "cities": data["ORG"],
    "dates" : data["DATE"],
    "phones" : phone,
    "emails" : emails
}

print(json.dumps(final_output_data))

{"names": ["25.50", "299.00", "32.99", "6.50", "363.99", "400.00", "36.01"], "prices": ["25.50", "299.00", "32.99", "6.50", "363.99", "400.00", "36.01"], "cities": ["1xT-Shirt", "Bank Card FI IIE IE I91\nApprovaltt"], "dates": ["01-02-2018", "192107"], "phones": [], "emails": []}
